In [ ]:
#using T4 GPU Person identity detection model
!pip install dlib
!pip install numpy
!pip install matplotlib
!pip install opencv-python
!pip install face_recognition
!pip install numpy scikit-learn matplotlib

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 100.1/100.1 MB 6.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for face-recognition-models: filename=face_recognition_models-0.3.0-py2.py3-none-any.whl size=100566172 sha256=24b71c42a1942fbb4451ad2d60187c6e131fc7192f9da12fd1239c041f291dbe
  Stored in directory: /root/.cache/pip/wheels/7a/eb/cf/e9eced74122b679557f597bb7c8e4c739cfcac526db1fd523d
Successfully built face-recognition-models


In [ ]:
import cv2
import dlib
import cv2
import numpy as np
import numpy as np
from PIL import Image
import face_recognition
from google.colab import files
from google.colab import drive
import matplotlib.pyplot as plt
from google.colab.patches import cv2_imshow

In [ ]:
#Mount Google Drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
# Load the Haar Cascade classifier for face detection
face_cascade = cv2.CascadeClassifier('/content/drive/MyDrive/haarcascade_frontalface_default.xml')

detector = dlib.get_frontal_face_detector()
predictor = dlib.shape_predictor("/content/drive/MyDrive/shape_predictor_68_face_landmarks.dat")

In [ ]:
def compare(img1,img2):
  gray_image1 = cv2.cvtColor(img1, cv2.COLOR_BGR2GRAY)
  gray_image2 = cv2.cvtColor(img2, cv2.COLOR_BGR2GRAY)

  # Detect faces in the first image
  faces1 = face_cascade.detectMultiScale(gray_image1, scaleFactor=1.1, minNeighbors=5, minSize=(30, 30))
  faces2 = face_cascade.detectMultiScale(gray_image2, scaleFactor=1.1, minNeighbors=5, minSize=(30, 30))

  if len(faces1) <= 0:
    print("No faces found in the first image.")
    return 0
  elif len(faces2) <= 0:
    print("No faces found in the second image.")
    return 0
  else:
    #extract face 1
    largest_face1 = max(faces1, key=lambda f: f[2] * f[3])
    x1, y1, w1, h1 = largest_face1
    extracted_face1 = img1[y1:y1+h1, x1:x1+w1]

    #extract face 2
    largest_face2 = max(faces2, key=lambda f: f[2] * f[3])
    x2, y2, w2, h2 = largest_face2
    extracted_face2 = img2[y2:y2+h2, x2:x2+w2]

    # cv2_imshow(extracted_face1)
    # cv2_imshow(extracted_face2)

    cv2.imwrite('/content/drive/MyDrive/Data/extracted_face1.jpg', extracted_face1)
    cv2.imwrite('/content/drive/MyDrive/Data/extracted_face2.jpg', extracted_face2)


    try:
        encoded_face1 = face_recognition.load_image_file('/content/drive/MyDrive/Data/extracted_face1.jpg')
        face_encodings1 = face_recognition.face_encodings(encoded_face1)[0]

        encoded_face2 = face_recognition.load_image_file('/content/drive/MyDrive/Data/extracted_face2.jpg')
        face_encodings2 = face_recognition.face_encodings(encoded_face2)[0]
    except Exception as e:
        print("An error occurred:", str(e))
        return 0

    if len(face_encodings1)==0:
      print("No face detected in images1.")
      return 0
    elif len(face_encodings2)==0:
      print("No face detected in images2.")
      return 0
    else:
      face_distance = np.linalg.norm(np.array(face_encodings1) - np.array(face_encodings2))
      threshold = 0.6

      if face_distance < threshold:
          print("The faces are a match!")
          return 1
      else:
          print("The faces are not a match.")
          return 0



In [ ]:
def calculate_eye_distance(img):
    image_np = np.array(img)
    faces = detector(image_np)

    # Ensure only one face is detected
    if len(faces) != 1:
        print("Error: Exactly one face should be detected.")
        return None
    else:
        landmarks = predictor(image_np, faces[0])

        left_eye = (landmarks.part(36).x, landmarks.part(36).y)
        right_eye = (landmarks.part(45).x, landmarks.part(45).y)

        distance_pixels = np.linalg.norm(np.array(left_eye) - np.array(right_eye))

        average_interocular_distance_mm = 65
        conversion_factor = average_interocular_distance_mm / distance_pixels
        print("Distance in Pixels: ",distance_pixels)
        distance_mm = distance_pixels * conversion_factor
        return distance_mm

inp_img = cv2.imread('/content/drive/MyDrive/Face_2.jpg')
distance = calculate_eye_distance(inp_img)
if distance is not None:
    print("Distance:", distance, "mm")


Distance in Pixels:  187.06683297688022
Distance: 65.0 mm


In [ ]:
# Load the first image
image1 = cv2.imread('/content/drive/MyDrive/Colab Notebooks/testing/face/f1.jpg')
image2 = cv2.imread('/content/drive/MyDrive/Colab Notebooks/testing/id/id1.jpg')
compare(image1,image2)

The faces are a match!


1

In [ ]:
predicted_labels = []
actual_labels =    [1,1,1,1,1,1]

# Load the first image
f1 = cv2.imread('/content/drive/MyDrive/Colab Notebooks/testing/face/f1.jpg')
id1 = cv2.imread('/content/drive/MyDrive/Colab Notebooks/testing/id/id1.jpg')
print("case 1 ",end="")
predicted_labels.append(compare(f1,id1))


# Load the second image
f2 = cv2.imread('/content/drive/MyDrive/Colab Notebooks/testing/face/f2.jpg')
id2 = cv2.imread('/content/drive/MyDrive/Colab Notebooks/testing/id/id2.jpg')
print("case 2 ",end="")
predicted_labels.append(compare(f2,id2))

# Load the third image
f3 = cv2.imread('/content/drive/MyDrive/Colab Notebooks/testing/face/f3.jpg')
id3 = cv2.imread('/content/drive/MyDrive/Colab Notebooks/testing/id/id3.jpg')
print("case 3 ",end="")
predicted_labels.append(compare(f3,id3))

# Load the fourth image
f4 = cv2.imread('/content/drive/MyDrive/Colab Notebooks/testing/face/f4.jpg')
id4 = cv2.imread('/content/drive/MyDrive/Colab Notebooks/testing/id/id4.jpg')
print("case 4 ",end="")
predicted_labels.append(compare(f4,id4))

# Load the fifth image
f5 = cv2.imread('/content/drive/MyDrive/Colab Notebooks/testing/face/f5.jpg')
id5 = cv2.imread('/content/drive/MyDrive/Colab Notebooks/testing/id/id5.jpg')
print("case 5 ",end="")
predicted_labels.append(compare(f5,id5))

# Load the sixth image
f6 = cv2.imread('/content/drive/MyDrive/Colab Notebooks/testing/face/f6.jpg')
id6 = cv2.imread('/content/drive/MyDrive/Colab Notebooks/testing/id/id6.jpg')
print("case 6 ",end="")
predicted_labels.append(compare(f6,id6))

# Convert lists to numpy arrays for easier manipulation
predicted_labels = np.array(predicted_labels)
actual_labels = np.array(actual_labels)

print("Actual Labels: ",actual_labels)
print("Predicted Labels: ",predicted_labels)

case 1 The faces are a match!
case 2 An error occurred: list index out of range
case 3 The faces are a match!
case 4 The faces are not a match.
case 5 An error occurred: list index out of range
case 6 An error occurred: list index out of range
Actual Labels:  [1 1 1 1 1 1]
Predicted Labels:  [1 0 1 0 0 0]


In [ ]:
from sklearn.metrics import confusion_matrix

conf_matrix = confusion_matrix(actual_labels, predicted_labels)

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

# Create a heatmap for the confusion matrix
plt.figure(figsize=(8, 6))
sns.heatmap(conf_matrix, annot=True, fmt='d', cmap='Blues', cbar=False,
            xticklabels=['Different', 'Same'], yticklabels=['Different', 'Same'])
plt.xlabel('Predicted Labels')
plt.ylabel('Actual Labels')
plt.title('Confusion Matrix')
plt.show()


In [ ]:
print("Face 1: ")
cv2_imshow(f1)
distance = calculate_eye_distance(inp_img)
if distance is not None:
    print("Distance:", distance, "mm")

print("Face 2: ")
cv2_imshow(f2)
distance = calculate_eye_distance(inp_img)
if distance is not None:
    print("Distance:", distance, "mm")

print("Face 3: ")
cv2_imshow(f3)
distance = calculate_eye_distance(inp_img)
if distance is not None:
    print("Distance:", distance, "mm")

print("Face 4: ")
cv2_imshow(f4)
distance = calculate_eye_distance(inp_img)
if distance is not None:
    print("Distance:", distance, "mm")

print("Face 5: ")
cv2_imshow(f5)
distance = calculate_eye_distance(inp_img)
if distance is not None:
    print("Distance:", distance, "mm")

print("Face 6: ")
cv2_imshow(f6)
distance = calculate_eye_distance(inp_img)
if distance is not None:
    print("Distance:", distance, "mm")